In [1]:
from utils import ReadTree
import pandas as pd
import numpy as np

In [2]:
# Parameters
nodsegmax = 15
outletPressure = 22.9
qProx = 9.41e-6 # cl/s

In [3]:
treeData, treeConnectivity = ReadTree('SmallNet_0.cco')
print(treeData[-1])
treeData.sort()
treeConnectivity.sort()
Ids = [v[0] for v in treeData]

Reading SmallNet_0.cco
	Number of vessels in the tree: 415 

[382, 6.317297353180776, 0.5357142857142857, 60.66758241067107, -756.6666666666667, -228.33333333333326, 0.0, -740.0, -169.9999999999999, 0.0, -1]


In [4]:
treeDict = dict()
rootTreeDict = dict()
for v in treeData:
    if v[-1] > -2:
        treeDict[v[0]] = v[1:] # Vessels to keep in the final 
    else:
        rootTreeDict[v[0]] = v[1:]
print(f'Found {len(treeDict)} segments in the FOV and {len(rootTreeDict)} outside the FOV.')

Found 363 segments in the FOV and 52 outside the FOV.


In [5]:
# Identify the inlet vessels
idInletSeg = []
for v in treeConnectivity:
    if v[0] in treeDict:
        vStage = treeDict[v[0]][-1]
        parentId = v[1]
        if (parentId in rootTreeDict) and rootTreeDict[parentId][-1]==-2:
            idInletSeg.append(v[0])
        
print(f'Found {len(idInletSeg)} inlet segments.')

Found 10 inlet segments.


In [6]:
# Identify terminal (outlet) vessels
idOutletSeg = []
for v in treeConnectivity:
    if len(v[-1])==0 and v[0] in treeDict:
        idOutletSeg.append(v[0])
print(f'Found {len(idOutletSeg)} outlet segments with IDs: {idOutletSeg}.')

Found 110 outlet segments with IDs: [45, 66, 80, 108, 185, 201, 219, 224, 229, 233, 235, 239, 241, 253, 255, 256, 257, 259, 260, 263, 264, 265, 267, 270, 271, 273, 275, 277, 279, 281, 283, 289, 291, 293, 295, 296, 297, 298, 299, 300, 301, 303, 305, 306, 307, 309, 312, 313, 314, 315, 316, 317, 318, 323, 326, 328, 333, 335, 336, 338, 340, 341, 343, 344, 345, 346, 348, 350, 352, 353, 356, 357, 360, 361, 362, 363, 365, 367, 370, 373, 375, 377, 381, 382, 385, 386, 387, 389, 390, 391, 392, 393, 394, 395, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 409, 411, 412, 413, 414].


In [17]:
dflist = [[v[0], 2*v[1], v[3], v[2], v[4], v[5], v[6], v[7], v[8],v[9], treeConnectivity[i][1], treeConnectivity[i][0], 
           v[0] in idInletSeg, v[0] in idOutletSeg, v[0] in treeDict] for i,v in enumerate(treeData)]
df = pd.DataFrame(dflist, columns=['Id', 'Diameter', 'Length', 'Flow', 'xprox','yprox','zprox',
                                   'xdist', 'ydist','zdist', 'From', 'To', 'Inlet', 'Outlet', 'InFOV'])
df = df.set_index('Id')
df = df[df.InFOV==True]
df = df.reset_index(drop=False)
df

,Id,Diameter,Length,Flow,xprox,yprox,zprox,xdist,ydist,zdist,From,To,Inlet,Outlet,InFOV
0,23,41.011833,10.540926,7.500000,1200.000000,1480.000000,0.0,1203.333333,1470.000000,0.0,22,23,True,False,True
1,24,38.500232,94.339811,5.892857,1220.000000,1420.000000,0.0,1270.000000,1340.000000,0.0,252,24,False,False,True
2,25,38.500232,18.633900,5.892857,1270.000000,1340.000000,0.0,1278.333333,1323.333333,0.0,24,25,False,False,True
3,26,37.765256,76.157731,5.357143,1320.000000,1240.000000,0.0,1350.000000,1170.000000,0.0,268,26,False,False,True
4,27,37.765256,107.703296,5.357143,1350.000000,1170.000000,0.0,1390.000000,1070.000000,0.0,26,27,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,410,11.299268,37.396978,0.535714,-834.722222,196.111111,0.0,-800.000000,210.000000,0.0,330,410,False,False,True
359,411,15.726664,215.763151,0.535714,-154.453224,-587.102543,0.0,-26.351318,-413.483307,0.0,371,411,False,True,True
360,412,13.516831,103.341352,0.535714,-154.453224,-587.102543,0.0,-59.531434,-546.245612,0.0,371,412,False,True,True
361,413,17.798613,114.816513,0.535714,-501.441263,-280.483831,0.0,-612.793863,-308.473650,0.0,408,413,False,True,True


In [18]:
nodesList = dict()
nodesMapping = dict()
nodeName  = 1
for index, seg in df.iterrows():
    nodePos = [seg.xdist, seg.ydist, seg.zdist]
    nodesList[nodeName] = nodePos
    #newToId = treeConnectivity[seg.Id][-1]
    #df.at[index,'To'] = newToId
    nodesMapping[int(seg.From)] = nodeName # The name of the starting node
    nodeName+=1
for index, seg in df.loc[df.Outlet==True].iterrows():
    ## This needs to map the To column of outlet nodes with a new node name 
    df.at[index,'To'] = nodeName
    nodeName+=1
print(f'Found {len(nodesList)} nodes in the tree.')

Found 363 nodes in the tree.


In [19]:
df.From = df.From.map(nodesMapping)
#df.To   = df.To.map(nodesMapping)
df.index +=1
df

,Id,Diameter,Length,Flow,xprox,yprox,zprox,xdist,ydist,zdist,From,To,Inlet,Outlet,InFOV
1,23,41.011833,10.540926,7.500000,1200.000000,1480.000000,0.0,1203.333333,1470.000000,0.0,1,23,True,False,True
2,24,38.500232,94.339811,5.892857,1220.000000,1420.000000,0.0,1270.000000,1340.000000,0.0,2,24,False,False,True
3,25,38.500232,18.633900,5.892857,1270.000000,1340.000000,0.0,1278.333333,1323.333333,0.0,3,25,False,False,True
4,26,37.765256,76.157731,5.357143,1320.000000,1240.000000,0.0,1350.000000,1170.000000,0.0,4,26,False,False,True
5,27,37.765256,107.703296,5.357143,1350.000000,1170.000000,0.0,1390.000000,1070.000000,0.0,5,27,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,410,11.299268,37.396978,0.535714,-834.722222,196.111111,0.0,-800.000000,210.000000,0.0,359,410,False,False,True
360,411,15.726664,215.763151,0.535714,-154.453224,-587.102543,0.0,-26.351318,-413.483307,0.0,361,470,False,True,True
361,412,13.516831,103.341352,0.535714,-154.453224,-587.102543,0.0,-59.531434,-546.245612,0.0,361,471,False,True,True
362,413,17.798613,114.816513,0.535714,-501.441263,-280.483831,0.0,-612.793863,-308.473650,0.0,363,472,False,True,True


In [10]:
f = open('Network.dat', 'w')
f.write('Network for patient1, sim0, stage0\n')
f.write('3000 3000 20\t box dimensions in microns\n')
#f.write(f'{df.xdist.max()-df.xprox.min()} {df.ydist.max()-df.yprox.min()} 20\n')
f.write('150 150 5\t number of tissue points in x,y,z directions\n')
f.write(f'{df.Length.max()}\t outer bound distance\n')
f.write(f'{df.Length.max()/nodsegmax}\t max. segment length.  (5. in test case)\n')
f.write(f'{nodsegmax}  \t nodsegm, max. allowed number of segments per node\n')
f.write(f'{int(df.shape[0])}\t total number of segments\n')
f.write('name\t type\t from\t to\t diam\t\t flow\t\t HD\n')

39

In [11]:
for index, row in df.iterrows():
    if row.Outlet:
        f.write(f'{index}\t 5\t {int(row.From)}\t {int(row.To)}\t {row.Diameter}\t\t {row.Flow}\t 0.4 *\n')
    else:
        ToNode = df.at[row.]
        f.write(f'{index}\t 5\t {int(row.From)}\t {int(row.To)}\t {row.Diameter}\t\t {row.Flow}\t 0.4 *\n')

ValueError: cannot convert float NaN to integer

In [ ]:
f.write(f'{len(nodesList)} total number of nodes\n')
f.write('name\t x\t y\t z\n')
for IdNode in nodesList:
    x,y,z = nodesList[IdNode]
    f.write(f'{IdNode}\t {x}\t {y}\t 10*\n')

In [ ]:
f.write(f'{len(idInletSeg)+len(idOutletSeg)} total number of boundary nodes\n')
f.write('node\t bctyp\t press/flow\t HD\t PO2\t solute\n')

# Inlet nodes -> specified flow
inletSegDF = df.loc[df.Inlet==True]
print(f'Found {len(inletSegDF)} inlets nodes: {np.array(inletSegDF.From).astype(int)}.')
for idSeg, seg in inletSegDF.iterrows():
    idNode = seg.From
    f.write(f'{idNode}\t 2\t {qProx}\t 0.4\t 40.0\t *\n')

In [ ]:
# Outlet nodes -> specified pressure
outletSegDF = df.loc[df.Outlet==True]
print(f'Found {len(outletSegDF)} inlets nodes: {np.array(outletSegDF.From).astype(int)}.')
for idSeg, seg in outletSegDF.iterrows():
    idNode = seg.To
    f.write(f'{idNode}\t 0\t {outletPressure}\t 0.4\t 40.0\t *\n')


In [ ]:
f.close()

In [ ]:
df.loc[df.From==4]

In [ ]:
df.loc[df.To==4]

In [ ]:
treeConnectivity.sort()
print(df.loc[df.Id==273].To.values[0])
print(df.loc[df.Id==274])
print(treeConnectivity[273])
print(treeConnectivity[274])